In [2]:
import cv2
import numpy as np
from IPython.display import Image, display
from matplotlib import pyplot as plt

In [4]:
#function to detect keypoints and matches
def detect_and_match_keypoints(img1, img2):
    sift = cv2.SIFT_create()
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # Detecting key points and descriptors
    kp1, des1 = sift.detectAndCompute(gray1, None)
    kp2, des2 = sift.detectAndCompute(gray2, None)

    # Using FLANN-based matcher
    index_params = dict(algorithm=1, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    matches = flann.knnMatch(des1, des2, k=2)

    # Filtering good matches using Lowe’s ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)

    return kp1, kp2, good_matches

# function to compute homography matrix using matched keypoints using RANSAC
def compute_homography(kp1, kp2, matches):
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    H, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)

    return H

#function to stitch the images
def stitch_images(images):
    base_image = images[1]
    img = images[0]
    for i in range(0, len(images)):
        if (i!=len(images)-1):
            base_image = images[i+1]
            kp1, kp2, good_matches = detect_and_match_keypoints(base_image, img)
            H = compute_homography(kp1, kp2, good_matches)
            h, w = base_image.shape[:2]

            # Warping the second image onto the first
            panorama = cv2.warpPerspective(img, H, (w * 2, h))  # Wider canvas
            # Overlaying the first image onto the warped one
            panorama[0:h, 0:w] = base_image

            # Crop black areas
            base_image = crop_black_area(panorama)
            img = base_image


    return base_image

#croping the black region
def crop_black_area(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        x, y, w, h = cv2.boundingRect(contours[0])  
        cropped_image = image[y:y+h, x:x+w]  
        return cropped_image
    
    return image  

if __name__ == "__main__":
    img1 = cv2.imread("input/img1.jpeg")  
    img2 = cv2.imread("input/img2.jpeg")
    img3 = cv2.imread("input/img3.jpeg")
    images = [img1, img2, img3]  

    panorama = stitch_images(images)
    cv2.imwrite("output/stitched_panorama.jpg", panorama)

